In [ ]:
pip install mplfinance

### Exploratory Data Analysis

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt


### Read in the files 

In [ ]:
oil=pd.read_csv('./data/oil_dfinal.csv')

In [ ]:
tech=pd.read_csv('./data/tech_dfinal.csv')

In [ ]:
events=pd.read_csv('./data/final_events.csv')

### Oil cleaning

In [ ]:
oil['Date']=oil.Date.str[:10]

In [ ]:
oil["Date"]=pd.to_datetime(oil['Date'])

In [ ]:
oil.dtypes

In [ ]:
oil

In [ ]:
oil.dtypes

In [ ]:
#drop columns not being used
to_drop=['Volume','Dividends','Stock Splits']
oil=oil.drop(columns=to_drop)

In [ ]:
grouped_mean=oil.groupby('Ticker')['Open'].mean()

In [ ]:
grouped_mean=grouped_mean.to_frame().reset_index()

In [ ]:
grouped_mean

In [ ]:
plt.bar('Ticker','Open',data=grouped_mean)
plt.title('Mean Open For Oil Companies')
plt.xlabel('Ticker')
plt.ylabel('Price')
plt.show()

In [ ]:
oilb

### Events cleaning 

In [ ]:
events=events.loc[(events['Year']>2000) & (events["Year"]<=2020)]

In [ ]:
events['Date']=pd.to_datetime(events['Date'])

In [ ]:
events=events[events['Date'].notnull()]

In [ ]:
events['Type']=events['Type'].str.replace('Domestic  Terrorism','Domestic Terrorism')


In [ ]:
events['Type']=events['Type'].str.replace('Domestic Terrorism ','Domestic Terrorism')

In [ ]:
index_to_drop=657

In [ ]:
events=events.drop(index=index_to_drop)

In [ ]:
events.Type=events['Type'].str.replace('Accident ','Accident')

In [ ]:
events=events.rename(columns={'Date':'Event_date'})


In [ ]:
events=events.loc[:,['Fatalities','Year','Article','Type','Location','Event_date']]

In [ ]:
events.Fatalities=events.Fatalities.str.replace('2,996','2996')

In [ ]:
events.Fatalities=events.Fatalities.astype(int)

In [ ]:
events['month']=events['Event_date'].dt.month

### cleaned events df

In [ ]:
events.dtypes

In [ ]:

events.Type

In [ ]:
events.Type.loc[events['Type']=='Terrorism']='Int Terrorsim'

In [ ]:
events.Type

events.Fatalities=events.Fatalities.astype(int)


In [ ]:
#events['Type']=events['Type'].str.replace('Terrorism','Non Domestic')

In [ ]:
events['Location']=events['Location'].str.replace('Sutherland Springs, Texas','Domestic')

### Final Before event has occured

In [ ]:




rolling_window_size = '7D'

oilb = pd.DataFrame()

# Iterate through each historical event
for index, event_row in events.iterrows():
    event_date = event_row['Event_date']
    
    # Calculate the date range for 7 days before the event
    date_range_start = event_date - pd.Timedelta(days=7)
    
    # Filter oil DataFrame for the specified date range
    filtered_data = oil[(oil['Date'] >= date_range_start) & (oil['Date'] <= event_date)]
    
    # Add the 'Event_date' and 'Type' columns from events DataFrame to each row
    filtered_data['Event_date'] = event_date
    filtered_data['Type'] = event_row['Type']
    
    # Append the filtered data to the oilb DataFrame
    oilb = pd.concat([oilb, filtered_data])




In [ ]:
oilb=oilb.reset_index()

In [ ]:
oilb

## Lets Dive

1. Retrieve the averages of industry through Types 5-7 days before the event has occurred.

In [ ]:
av_oilb= oilb.groupby('Type').agg({'Open':'mean','High':'mean','Low':'mean','Close':'mean'})

2.Retrieve standard deviation industries before the event

In [ ]:
sd_oilb=oilb.groupby('Type').agg({'Open':'std','High':'std','Low':'std','Close':'std'})

In [ ]:
av_oilb=av_oilb.reset_index()
av_oilb

In [ ]:
sd_oilb=sd_oilb.reset_index()
sd_oilb

- candlestick graph doesnt really tell us much, the dates are not relevant at all in this situation.

In [ ]:
import mplfinance as mpf

mpf.plot(oilb,type='candle',style='charles')

- compares the av_close and open price for oil industry as a whole before events with the average based off of groups

In [ ]:

# Create a new DataFrame with 'Type', 'Open', and 'Close' columns
type_open_close_df = oilb[['Type', 'Open', 'Close']]

# Melt the DataFrame to a long format for plotting
melted_df = type_open_close_df.melt(id_vars='Type', value_vars=['Open', 'Close'], var_name='Price Type', value_name='Price')

# Set the style of seaborn
sns.set(style="whitegrid")

# Define colors to match the bar plot colors
colors = ['blue', 'red']

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Type', y='Price', hue='Price Type', data=melted_df)

# Create the line plot with matching colors
sns.lineplot(x='Type', y='Price', hue='Price Type', data=melted_df, palette=colors)

plt.title('Comparison of Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


Looks like the average doesnt fall too far out of line with groups.

In [ ]:


# Create a new DataFrame with 'Type', 'Open', and 'Close' columns
type_open_close_df = oilb[['Type', 'Open', 'Close']]

# Melt the DataFrame to a long format for plotting
melted_df = type_open_close_df.melt(id_vars='Type', value_vars=['Open', 'Close'], var_name='Price Type', value_name='Price')

# Set the style of seaborn
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Type', y='Price', hue='Price Type', data=melted_df)

plt.title('Comparison of Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



- Finding the diff in average price change for opening and closing prices before event.

In [ ]:

oilb['Price Change'] = oilb['Close'] - oilb['Open']


average_changes = oilb.groupby('Type')['Price Change'].mean()

average_changes

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named oilb with calculated 'Price Change' column

# Group by 'Type' and calculate the average of price changes
average_changes = oilb.groupby('Type')['Price Change'].mean().reset_index()

# Set the style of seaborn
sns.set(style="whitegrid")
sns.set_palette(color_palette)
# Create the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Type', y='Price Change', data=average_changes)

plt.title('Average Change in Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Average Price Change')
plt.xticks(rotation=45)

# Invert the y-axis
ax.invert_yaxis()

plt.tight_layout()
plt.show()


-calculate percent change between close and opening price

In [ ]:

oilb['Percent Change'] = ((oilb['Close'] - oilb['Open']) / oilb['Open']) * 100


In [ ]:




# Group by 'Type' and calculate the average of percent changes
average_percent_changes = oilb.groupby('Type')['Percent Change'].mean().reset_index()

average_percent_changes


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named oilb with calculated 'Percent Change' column

# Group by 'Type' and calculate the average of percent changes
average_percent_changes = oilb.groupby('Type')['Percent Change'].mean().reset_index()

# Set the style of seaborn
sns.set(style="whitegrid")
sns.set_palette(color_palette)
# Create the bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x='Type', y='Percent Change', data=average_percent_changes)

plt.title('Average Percent Change in Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Average Percent Change (%)')
plt.xticks(rotation=45)

# Invert the y-axis
ax.invert_yaxis()


plt.tight_layout()
plt.show()


* we can see that our events that occur 5-7 days before event have the drop over 3% 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have a DataFrame named sd_oilb with calculated standard deviations

# Define the colors to match the previous graphs
colors = ['blue', 'red']

# Set the style of seaborn
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Type', y='Open', data=sd_oilb, color=colors[0], label='Open Price SD')
sns.barplot(x='Type', y='Close', data=sd_oilb, color=colors[1], label='Close Price SD')

plt.title('Standard Deviations of Open and Close Prices by Type')
plt.xlabel('Type')
plt.ylabel('Standard Deviation')
plt.xticks(rotation=45)

plt.legend()
plt.tight_layout()
plt.show()


- above graph is showing sd of opening and closing prices. As we can see there is not votatlity between our prices.

### events after the event oil industry

In [ ]:
rolling_window_size = '7D'

oila = pd.DataFrame()

# Iterate through each historical event
for index, event_row in events.iterrows():
    event_date = event_row['Event_date']
    
    # Calculate the date range for 7 days after the event
    date_range_end = event_date + pd.Timedelta(days=7)
    
    # Filter oil DataFrame for the specified date range
    filtered_data = oil[(oil['Date'] > event_date) & (oil['Date'] <= date_range_end)]
    
    # Add the 'Event_date' and 'Type' columns from events DataFrame to each row
    filtered_data['Event_date'] = event_date
    filtered_data['Type'] = event_row['Type']
    
    # Append the filtered data to the oilb DataFrame
    oila = pd.concat([oila, filtered_data])

# Reset index of the resulting DataFrame
oila = oila.reset_index(drop=True)



In [ ]:
oila.columns

In [ ]:
av_oila= oila.groupby('Type').agg({'Open':'mean','High':'mean','Low':'mean','Close':'mean'})

In [ ]:
sd_oila=oila.groupby('Type').agg({'Open':'std','High':'std','Low':'std','Close':'std'})

In [ ]:
av_oila=av_oila.reset_index()

In [ ]:
av_oila

In [ ]:
sd_oila=sd_oila.reset_index()
sd_oila

In [ ]:
# Create a new DataFrame with 'Type', 'Open', and 'Close' columns
type_open_close_dfa= oila[['Type', 'Open', 'Close']]

# Melt the DataFrame to a long format for plotting
melted_dfa = type_open_close_dfa.melt(id_vars='Type', value_vars=['Open', 'Close'], var_name='Price Type', value_name='Price')

# Set the style of seaborn
sns.set(style="whitegrid")

# Define colors to match the bar plot colors
colors = ['blue', 'red']

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Type', y='Price', hue='Price Type', data=melted_dfa)

# Create the line plot with matching colors
sns.lineplot(x='Type', y='Price', hue='Price Type', data=melted_dfa, palette=colors)

plt.title('Comparison of Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Create a new DataFrame with 'Type', 'Open', and 'Close' columns
type_open_close_df = oila[['Type', 'Open', 'Close']]

# Melt the DataFrame to a long format for plotting
melted_df = type_open_close_df.melt(id_vars='Type', value_vars=['Open', 'Close'], var_name='Price Type', value_name='Price')

# Set the style of seaborn
sns.set(style="whitegrid")

# Create the bar plot
plt.figure(figsize=(10, 6))
sns.barplot(x='Type', y='Price', hue='Price Type', data=melted_df)

plt.title('Comparison of Opening and Closing Prices by Type')
plt.xlabel('Type')
plt.ylabel('Price')
plt.legend()
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



-calculate percent change for after oil and difference in price

In [ ]:
oila['Price Change'] = oila['Close'] - oila['Open']

In [ ]:
oila['Percent Change'] = ((oila['Close'] - oila['Open']) / oila['Open']) * 100


In [ ]:
# Group by 'Type' and calculate the average of percent changes
average_percent_changesa = oila.groupby('Type')['Percent Change'].mean().reset_index()
aver
average_percent_changesa


In [ ]:
oila

#### effect size 

In [ ]:
oila

In [ ]:
oilb

In [ ]:
columns_to_use='Open'
mean_diff_oil=oila[columns_to_use].mean()-oilb[columns_to_use].mean()

In [ ]:
mean_diff_oil

In [ ]:
pooled_sd= np.sqrt((oilb[columns_to_use]**2+oila[columns_to_use]**2)/2)

In [ ]:
oila.shape

In [ ]:
oilb.shape

In [ ]:
oilb=oilb.drop(columns='Year')


In [ ]:
oila

In [ ]:
oilb